In [1]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project='your-project-ID')
import pprint

In [2]:
# Load urban boundaries (GAUL dataset) - Level 1
urban = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")

# Filter Tehran boundary (ADM1 level)
tehran = urban.filter(ee.Filter.eq('ADM1_NAME', 'Tehran'))
geometry = tehran.geometry()



# Load and filter Sentinel-2 imagery
s2 = ee.ImageCollection("COPERNICUS/S2")
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
             .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
             .filter(ee.Filter.bounds(geometry))

# Create median composite and clip to Tehran
composite = filtered.median().clip(geometry)

/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


In [3]:
# Function to compute NDVI and add it as a band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands(ndvi)

# Map the function over the collection
with_ndvi = filtered.map(add_ndvi)

# Create median composite of NDVI-enhanced collection
composite = with_ndvi.median()

# Extract NDVI band and clip to Tehran
ndvi_composite = composite.select('ndvi').clip(geometry)
# pprint.pprint(ndvi_composite.getInfo())

In [4]:
# Create a map using geemap
Map = geemap.Map()
# Center the map on Tehran
Map.centerObject(geometry, 8)

# NDVI color palette
palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']

# Visualization parameters for RGB
rgb_vis = {'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
Map.addLayer(composite, rgb_vis, 'Tehran Composite')

# NDVI visualization parameters
ndvi_vis = {'min': 0, 'max': 0.5, 'palette': palette}
Map.addLayer(ndvi_composite, ndvi_vis, 'NDVI')
Map

Map(center=[35.66879776649012, 51.475366200268944], controls=(WidgetControl(options=['position', 'transparent_…